# Notebook Lecture 5: Transfer Functions
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook covers the basics of response to exponential inputs, transfer functions, conversions between state-space models and transfer functions, and connections to Laplace transforms. 

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch

## Learning Objectives

In previous lectures, we've seen that this course addresses the control of dynamical systems. Two key course objectives are to understand how to analyze and synthesize systems to achieve specific goals. To do this effectively, we need to understand various ways of representing and analyzing dynamical systems mathematically.
In this notebook, we delve deeper into some fundamental concepts and techniques used in control systems analysis. We'll explore system responses and different system representations.
After completing this notebook, you should be able to:

- Understand and analyze a system's response to exponential inputs.
- Interpret and work with transfer functions.
- Convert between state-space models and transfer functions, and understand the benefits of each representation.
- Appreciate the role of Laplace transforms in control systems analysis and how they relate to system behavior.

### Import the packages:

The following cell imports the required packages. Run it before running the rest of the notebook.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from ipywidgets import interactive, FloatSlider
import ipywidgets as widgets
from IPython.display import display, Markdown, Latex
from scipy.fft import fft, fftfreq

# Motivation
In Lecture 4, we observed that analyzing the time response of a Linear Time-Invariant (LTI) system can be quite challenging. Their time response is given by:

$$y(t) = Ce^{At}x(0) + C\int_{0}^{t} e^{A(t-\tau)}Bu(\tau)d\tau + Du(t)$$

This formula provides a complete characterization of the output response of an LTI system. However, the convolution integral makes it difficult to analyze the system's behavior.

This raises the question: how can we effectively analyze the response of a system to more complex inputs?

To address this challenge, we will study the response of LTI systems to elementary inputs such as step and impulse inputs. These inputs are particularly easy to analyze because the input has the same form as the output and are very rich, meaning they can be used to describe any other input through linear combinations.


# 1. Exponential Inputs

In this section, we will study the response of an LTI system to an exponential input. The exponential input is a particularly interesting input because it is an eigenfunction of the system. This means that the output of the system will be a scaled version of the input.

## Definition of Exponential Input

An exponential input is defined as:

$$u(t) = e^{st}$$

where $s \in \mathbb{C}$ is a complex number.

### Special Cases:

1. **Real Exponential**: If $s$ is real, then $u(t) = e^{st}$ is a simple exponential function.

2. **Sinusoidal**: If $s = j\omega$ is purely imaginary, then $u(t) = e^{j\omega t}$ represents a sinusoidal signal:
   
   $$u(t) + u^*(t) = e^{j\omega t} + e^{-j\omega t} = 2\cos(\omega t)$$

3. **Exponentially Modulated Sinusoid**: If $s = \sigma + j\omega$, then:
   
   $$u(t) + u^*(t) = e^{\sigma t}e^{j\omega t} + e^{\sigma t}e^{-j\omega t} = 2e^{\sigma t}\cos(\omega t)$$

This represents a sinusoid with exponentially changing amplitude.

## System Response to Exponential Input

For an LTI system described by the state-space equations:

\begin{align}
\dot{x}(t) &= Ax(t) + Bu(t) \\\\
y(t) &= Cx(t) + Du(t)
\end{align}

The response to an exponential input $u(t) = e^{st}$ is given by:

$$y(t) = Ce^{At}x(0) + C\int_0^t e^{A(t-\tau)}Be^{s\tau} d\tau + De^{st}$$

This can be simplified to:

$$y(t) = \underbrace{Ce^{At}\left(x(0) - (sI - A)^{-1}B\right)}_{\text{Transient Response } \rightarrow 0 \text{ (if as. stable)}} + \underbrace{\left(C(sI - A)^{-1}B + D\right)e^{st}}_{\text{Steady-State Response}}$$

<small><small>
See Lecture 5 page 7 for the derivation
</small></small>

Where:
- The first term represents the transient response
- The second term represents the steady-state response

### Steady-State Response

For an asymptotically stable system, the transient response will converge to zero as $t \to \infty$. The steady-state response is then:

$$y_{ss}(t) = G(s)e^{st}$$

where $G(s) = C(sI - A)^{-1}B + D$ is the transfer function of the system.

This property makes exponential inputs particularly useful for analyzing LTI systems:

1. It allows us to easily compute the frequency response of the system by setting $s = j\omega$.
2. It forms the basis for Laplace transform analysis, as we can express any input as a sum of exponentials.
3. It helps us understand the concept of eigenvalues and eigenvectors in the context of dynamic systems.

### Example: First-Order System

Let's consider a first-order system with transfer function:

$$G(s) = \frac{1}{\tau s + 1}$$

Which could represent a simple RC circuit. We'll examine the system's response to a real exponential input $u(t) = e^{st}$.

In [ ]:
def plot_first_order_response(tau, sigma, omega, t_max):
    t = np.linspace(0, t_max, 1000)
    s = sigma + 1j * omega
    
    # System transfer function
    sys = signal.TransferFunction([1], [tau, 1])
    
    # Input signal (real part of the complex exponential)
    u = np.exp(sigma * t) * np.cos(omega * t)
    
    # Compute system response
    _, y, _ = signal.lsim(sys, U=u, T=t)
    
    # Compute steady-state response
    G_s = 1 / (tau * s + 1)
    gain = np.abs(G_s)
    phase = np.angle(G_s)
    y_ss = gain * np.exp(sigma * t) * np.cos(omega * t + phase)

    # Compute the transient response
    y_transient = y - y_ss
    
    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(t, u, label='Input')
    plt.plot(t, y, label='System Output')
    plt.plot(t, y_ss, '--', label='Steady-State Output')
    plt.plot(t, y_transient, '--', label='Transient Output')
    plt.title(f'First-Order System Response (τ = {tau:.2f})')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.grid(True)
    plt.show()

interactive_plot = interactive(plot_first_order_response, 
                                   tau=widgets.FloatSlider(min=0.1, max=5, step=0.1, value=1, description='τ:'),
                                   sigma=widgets.FloatSlider(min=-2, max=0, step=0.1, value=0, description='σ:'),
                                   omega=widgets.FloatSlider(min=-5, max=5, step=0.1, value=1, description='ω:'),
                                   t_max=widgets.FloatSlider(min=1, max=20, step=1, value=10, description='t_max:'),
                                   )
display(interactive_plot)


## More in Depth: Sinusoidal Inputs in LTI Systems

Sinusoidal inputs are particularly interesting in the study of Linear Time-Invariant (LTI) systems due to a fundamental property: the output of an LTI system to a sinusoidal input is also sinusoidal, with the same frequency as the input, but potentially different amplitude and phase. This property is crucial in understanding frequency response and forms the basis for many control system analysis techniques.

Consider a sinusoidal input of the form:

$u(t) = e^{j\omega} + e^{-j\omega} = 2\cos{\omega t}$

For an LTI system with transfer function $G(s)$, the output $y(t)$ can be expressed as:

$y(t) = G(j\omega)e^{j\omega t} + G(-j\omega)e^{-j\omega t}$

Using Euler's formula and the properties of complex conjugates, we can simplify this to:

$y(t) = A|G(j\omega)|\cos(\omega t + \angle G(j\omega))$

Where:
- $|G(j\omega)|$ is the magnitude of the transfer function at frequency $\omega$
- $\angle G(j\omega)$ is the phase angle of the transfer function at frequency $\omega$

### Key Observations

1. **Frequency Preservation**: The output frequency is identical to the input frequency $\omega$.
2. **Amplitude Modification**: The output amplitude is scaled by $|G(j\omega)|$.
3. **Phase Shift**: The output is phase-shifted by $\angle G(j\omega)$ relative to the input.


In [ ]:
def plot_lti_response(omega):
    # Define the LTI system
    num = [1]
    den = [1, 2, 1]  # (s + 1)^2
    sys = signal.TransferFunction(num, den)
    
    # Generate input and output
    t = np.linspace(0, 20, 1000)
    u = np.cos(omega * t)
    _, y, _ = signal.lsim(sys, u, t)
    
    # Calculate the theoretical gain and phase shift
    w, mag, _ = signal.bode(sys, w=[omega])
    gain = 10**(mag[0]/20)
    _, phase, _ = signal.bode(sys, w=[omega])
    phase_shift = np.deg2rad(-phase[0])
    
    # Plot
    plt.figure(figsize=(12, 8))
    plt.plot(t, u, label='Input', alpha=0.7)
    plt.plot(t, y, label='Output', alpha=0.7)
    plt.plot(t, gain * np.cos(omega * t + phase_shift), '--', label='Theoretical Output', alpha=0.7)
    
    plt.title(f'LTI System Response to Sinusoidal Input (ω = {omega:.2f} rad/s)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.grid(True)
    
    # Display gain and phase information
    plt.text(0.05, 0.95, f'Gain: {gain:.2f}\nPhase Shift: {np.rad2deg(phase_shift):.2f}°', 
             transform=plt.gca().transAxes, verticalalignment='top', 
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    
    plt.show()

# Create an interactive widget
interactive_plot = interactive(plot_lti_response, 
                               omega=FloatSlider(min=0.1, max=10, step=0.1, value=1, 
                                                 description='Frequency (rad/s)'))

# Display the interactive plot
display(interactive_plot)

# The Laplace Transform

The Laplace transform lets us move from the time domain to the frequency domain. This gives us the ability to work with LTI (Linear Time-Invariant) systems much more easily by transforming differential equations in the time domain into simple algebraic equations in the frequency domain.

In essence, the Laplace transform is a generalized Fourier transform. To get an intuition for the Laplace transform, let's quickly explain what the Fourier transform is and visualize it with a simple Python plot.

### Fourier Transform

The Fourier Transform converts a time-domain signal into its frequency components. This is particularly useful for analyzing periodic signals or any signal with oscillatory components. The Fourier transform is defined as:

$$
\mathcal{F}[f(t)] = F(\omega) = \int_{-\infty}^{\infty} f(t) e^{-j\omega t} dt
$$

In simpler terms, the Fourier Transform expresses a function in terms of its sine and cosine components. Let's visualize this with a simple example:


In [ ]:
def plot_fourier_transform(frequency):
    # Time settings
    t = np.linspace(0, 1, 1000, endpoint=False)  # time from 0 to 1 second
    # Create a sine wave signal with the input frequency
    signal = np.sin(2 * np.pi * frequency * t)

    # Perform the Fourier Transform
    signal_fft = fft(signal)
    frequencies = fftfreq(t.size, t[1] - t[0])

    # Plot the original signal (time domain)
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(t, signal)
    plt.title(f'Time Domain Signal (Frequency = {frequency} Hz)')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')

    # Plot the Fourier Transform (frequency domain)
    plt.subplot(1, 2, 2)
    plt.stem(frequencies[:t.size // 10], np.abs(signal_fft[:t.size // 10]), 'b', markerfmt=" ", basefmt="-b")
    plt.title('Frequency Domain (Fourier Transform)')
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Magnitude')

    plt.tight_layout()
    plt.show()

# Create an interactive plot with a slider for frequency
interactive_plot = interactive(plot_fourier_transform, 
                               frequency=widgets.FloatSlider(min=1, max=99, step=1, value=5, description='Frequency [Hz]'),
                              )
display(interactive_plot)


The Laplace transform of a signal $u$ is defined as:
$$\mathcal{L}[u] = U(s) = \int_0^\infty u(t) e^{-st} dt$$

And the inverse Laplace transform is defined as:
$$\mathcal{L}[U] = u(t) = \frac{1}{2\pi j}$$